In [580]:
import pandas as pd
import numpy as np
import os
from dbfread import DBF
import math

In [581]:
path = 'C:/Users/ScMa9004/Documents/Итоги визита/313/'
N = 313

In [582]:
new = pd.read_excel(path + 'new_' + str(N) + '.xls', skiprows = 1)

In [583]:
new['ocode'] = new['постоянный код новой  ТТ'].astype('str').str.slice(start = 4)

In [584]:
rep = pd.read_excel(path + 'отчет_по_смене_характеристик_' + str(N) + '_All.xls', skiprows = 1)

In [585]:
rep = rep.rename(columns = {'Код ТТ.1': 'ocode'})

In [586]:
# удаление из rep тех точек, которые есть в new (если тт и в rep и в new - для нас она new)
new['outlet'] = new['постоянный код новой  ТТ'].astype('str').str.slice(start = 4)
rep = rep[~rep['Код ТТ'].astype('str').isin(list(new['outlet'].astype('str')))]

In [587]:
####Светлана Артемова
sales_purches = pd.read_excel(path + 'метод сбора_fin_nov.xlsx')

In [588]:
msr = pd.read_csv('C:/Users/ScMa9004/Scripts/MSR.csv', sep = ';')

In [589]:
pc = pd.read_excel(path + 'panelcensus_' + str(N) + '.xlsx')

In [590]:
panrep = pd.read_excel(path + 'panel_report_00001_25.11.2019.xlsx', sheet_name = 'panel_report_00001_25.11.2019')
panrep = panrep[['outlet', 'bst']]
panrep = panrep.rename(columns = {'outlet': 'ocode'})

In [591]:
# данные о бустере в pc, new и rep неверные, а в panel report'e верные
pc = pd.merge(pc, panrep, on = 'ocode', how = 'left').rename(columns = {'bst_y': 'bst'})
new = pd.merge(new, panrep, on = 'ocode', how = 'left')
rep = pd.merge(rep, panrep, on = 'ocode', how = 'left')

In [596]:
# fill blank 'ocode' from sliced full name (2010XXXX -> XXXX)
pc.loc[(pc['ocode'].isna()), 'ocode'] = pc['Unnamed: 181'].str.slice(start = 4)

In [597]:
def tracking_system_input(N, new, rep, sales_purches, pc):
    
    df = pd.DataFrame()
    
    df['outlet'] = new['постоянный код новой  ТТ']
    df['pno'] = N
    df['region DA'] = new['регион DA']
    df['type'] = new['Тип ТТ']
    df['Type of info'] = new['type of info']
    df['Source of info'] = new['sourse of info']
    df['Interval for info'] = 'Full'
    df['Comments for DA'] = 'new'
    df['bst'] = new['bst']
    
    df['outlet'] = df['outlet'].astype('str').str.slice(start = 4)
    
    
    #fill blank type of info and source of info in new from pc
    for i in df['outlet'][df['Type of info'].isna()]:
        df.loc[df['outlet'] == i, 'Type of info'] = pc['purchsales'][pc['ocode'] == i].iloc[0]
    for i in df['outlet'][df['Source of info'].isna()]:
        df.loc[df['outlet'] == i, 'Source of info'] = pc['moc'][pc['ocode'] == i].iloc[0]
        
    df.loc[df['Type of info'] == 1, 'Type of info'] = 'purchase'
    df.loc[df['Type of info'] == 2, 'Type of info'] = 'sales'
    
    df.loc[df['Source of info'] == 4, 'Source of info'] = 'oral'
    df.loc[df['Source of info'].isin([1, 2, 3]), 'Source of info'] = 'documented'
    
    rep = rep.rename(columns = {'Регион': 'region DA', 'Код ТТ': 'outlet', 'Тип ТТ': 'old_type', 'Тип ТТ.1': 'type',\
                            'Код бустера': 'old_bst'})
    
    rep['Comments for DA'] = np.nan
    
    rep.loc[(rep['old_type'] != rep['type']) &\
        ~((rep['old_type'].isin(['OLA', 'OME']) & rep['type'].isin(['OLA', 'OME'])) |\
        (rep['old_type'].isin(['OKS', 'OPV']) & rep['type'].isin(['OKS', 'OPV'])) |\
        (rep['old_type'].isin(['OMM', 'OMS']) & rep['type'].isin(['OMM', 'OMS'])) |\
        (rep['old_type'].isin(['DKM', 'DKP', 'DKC']) & rep['type'].isin(['DKM', 'DKP', 'DKC']))),\
        'Comments for DA'] = 'смена типа'
    
    rep.loc[(rep['old_bst'] != rep['bst']) & (rep['old_bst'] == 3) &
            (rep['bst'].isin([1, 4, 501])), 'Comments for DA'] = 'смена бустера'
    rep.loc[(rep['old_bst'] != rep['bst']) & (rep['old_bst'] == 4) &
            (rep['bst'].isin([1, 3, 501])), 'Comments for DA'] = 'смена бустера'
    rep.loc[(rep['old_bst'] != rep['bst']) & (rep['old_bst'] == 501) &
            (rep['bst'] == 1), 'Comments for DA'] = 'смена бустера'
    rep.loc[(rep['old_bst'] != rep['bst']) & (rep['old_bst'] == 998) &
            (rep['bst'] == 999), 'Comments for DA'] = 'смена бустера'
    rep.loc[(rep['old_bst'] != rep['bst']) & (rep['old_bst'] == 999) &
            (rep['bst'] == 998), 'Comments for DA'] = 'смена бустера'
    
    rep = rep[rep['Comments for DA'].notna()]
    
    rep['pno'] = N
    rep['Interval for info'] = 'Full'
    
    sales_purches = sales_purches.rename(columns = {'shop': 'outlet'})
    sales_purches['outlet'] = sales_purches['outlet'].astype('str').str.slice(start = 4)
    sales_purches.loc[sales_purches['total_sales_lines'] != 0, 'Type of info'] = 'sales'
    sales_purches.loc[sales_purches['total_pur_lines'] != 0, 'Type of info'] = 'purchase'
    sales_purches.loc[(sales_purches['total_pur_lines'] != 0) &
                      (sales_purches['total_sales_lines'] != 0), 'Type of info'] = 'sales & purchase'
    
    
    # если нет данных в файле Светланы Артемовой - информация берется из pc
    # здесь можно сделать лучше, не ища наличие кодов тт, а просто беря их из pc
    sales_purches.loc[(sales_purches['total_pur_lines'] == 0) &
                (sales_purches['total_sales_lines'] == 0) &
                (sales_purches['outlet'].astype('str').isin(list(pc['ocode'].astype('str')[pc['ocode'].astype('str').isin\
                (list(sales_purches['outlet'].astype('str')[(sales_purches['total_pur_lines'] == 0) &
                (sales_purches['total_sales_lines'] == 0)])) & (pc['purchsales'] == 1)]))), 'Type of info'] = 'purchase'
    sales_purches.loc[(sales_purches['total_pur_lines'] == 0) &
                (sales_purches['total_sales_lines'] == 0) &
                (sales_purches['outlet'].astype('str').isin(list(pc['ocode'].astype('str')[pc['ocode'].astype('str').isin\
                (list(sales_purches['outlet'].astype('str')[(sales_purches['total_pur_lines'] == 0) &
                (sales_purches['total_sales_lines'] == 0)])) & (pc['purchsales'] == 2)]))), 'Type of info'] = 'sales'
    
    
    
    sales_purches.loc[(sales_purches['s_other_count'] /\
                       (sales_purches['s_other_count'] + sales_purches['s_Oral_count']) >= 0.8) &
                      (sales_purches['Type of info'] == 'sales'), 'Source of info'] = 'documented'
    sales_purches.loc[(sales_purches['Source of info'].isna()) &
                      (sales_purches['Type of info'] == 'sales'), 'Source of info'] = 'oral'

    sales_purches.loc[(sales_purches['pu_invoice_count'] + sales_purches['pu_other_count'] + sales_purches['pu_oral_count']) /\
                      (sales_purches['pu_oral_count']) >= 0.8, 'Source of info'] = 'documented'
    sales_purches.loc[(sales_purches['Source of info'].isna()) &
                      ((sales_purches['Type of info'] == 'purchase')), 'Source of info'] = 'oral'
    
    #sales_purches['Source of info'] = sales_purches['Source of info'].fillna('???')
    
    sales_purches = sales_purches[['outlet', 'Source of info', 'Type of info']]
    
    rep = rep.merge(sales_purches, on = 'outlet', how = 'inner')
    
    rep = rep[['outlet', 'pno', 'region DA', 'type', 'Type of info', 'Source of info',
           'Interval for info', 'Comments for DA', 'bst']]
    
    df = pd.concat([df, rep], sort = False).reset_index(drop = True)
    
    return df

In [598]:
df = tracking_system_input(N, new, rep, sales_purches, pc)

In [599]:
msr['outlet'] = msr['SMS_ID'].astype('str').str.slice(start = 4)

In [600]:
msr['cell'] = msr['MAS'].str.slice(start = 8).astype('int')

In [601]:
df = pd.merge(df, msr, how = 'left', on = ['outlet'])

In [602]:
df['status cha'] = ''
df['Chainact'] = ''

In [603]:
df['RU_CHA'] = df['RU_CHA'].replace('OTHER', np.nan)

In [604]:
for index,row in df.iterrows():
# fill om
    if (row['type'] == ('OKP')) | (row['type'] == ('OTK')) | (row['type'][0] == ('D')) | (row['type'][0] == ('F')):
        df.at[index,'om'] = 1
    else:
        df.at[index,'om'] = 0

# fill Chainact
    if isinstance(df['RU_CHA'], str) :
        df.at[index,'Chainact'] = '1'
    else:
        df.at[index,'Chainact'] = '0'

# fill status cha        
    if isinstance(df['RU_CHA'], str) :
        df.at[index,'status cha'] = 'check'
    else:
        df.at[index,'status cha'] = 'OK'

In [605]:
result = df

# COCA_PEPSI

In [606]:
wp1_c_p = pc[['ocode', 'coca', 'pepsi']].rename(columns = {'ocode': 'outlet', 'coca': 'COCA', 'pepsi': 'PEPSI'})

In [607]:
#wp1 = pd.DataFrame(DBF('C:/Users/ScMa9004/Documents/Итоги визита/313/wp1.dbf', load=True))
#wp1_c_p = wp1[['OCODE','COCA','PEPSI']]
#wp1_c_p = wp1_c_p.rename(columns={'OCODE': 'outlet'})

In [608]:
st_rum_c_p = msr.loc[(msr['INDEX_ID'] == 1000296) &
                     msr['STORE_TYPE'].isin(['OKS','OLA','OME','OMM', 'OMS', 'OPV', 'OSM', 'OSU'])]

In [609]:
wstatus_cell_coca_pepsi = pd.merge(st_rum_c_p, wp1_c_p, how = 'left', on = ['outlet'])

In [610]:
wstatus_cell_coca_pepsi['excl_c_p'] = ''

In [611]:
# fill excl_c_p
for index,row in wstatus_cell_coca_pepsi.iterrows():
    if (row['COCA'] == 1 and row['PEPSI'] == 0):
        wstatus_cell_coca_pepsi.at[index,'excl_c_p'] = 'coca'
        
    if (row['COCA'] == 0 and row['PEPSI'] == 1):
        wstatus_cell_coca_pepsi.at[index,'excl_c_p'] = 'pepsi'
        
    if (row['COCA'] == 1 and row['PEPSI'] == 1):
        wstatus_cell_coca_pepsi.at[index,'excl_c_p'] = 'blank'
        
    if (row['COCA'] == 0 and row['PEPSI'] == 0):
        wstatus_cell_coca_pepsi.at[index,'excl_c_p'] = 'blank'

In [612]:
# filling 'm'
wstatus_cell_coca_pepsi['m'] = ''
wstatus_cell_coca_pepsi['m'] = wstatus_cell_coca_pepsi['STORE_TYPE'] + '-' + wstatus_cell_coca_pepsi['TRADE_TYPE']

In [613]:
# filling channel
wstatus_cell_coca_pepsi['channel'] = ''

for index,row in wstatus_cell_coca_pepsi.iterrows():
    if (row['CITYCODE'] > 2 and row['TRADE_TYPE'] == 'TT'):
        wstatus_cell_coca_pepsi.at[index,'channel'] = 'TT'
        
    if (row['CITYCODE'] > 2 and row['TRADE_TYPE'] == 'MT'):
        wstatus_cell_coca_pepsi.at[index,'channel'] = 'MT'
        
    if (row['CITYCODE'] <= 2 and row['m'] in ('OKS-TT','OPV-TT')):
        wstatus_cell_coca_pepsi.at[index,'channel'] = 'Impulse stores'
    
    if (row['CITYCODE'] <= 2 and row['m'] == 'OSU-MT'):
        wstatus_cell_coca_pepsi.at[index,'channel'] = 'Supermarkets'
        
    if (row['CITYCODE'] <= 2 and row['m'] in ('OLA-MT', 'OME-MT', 'OMM-MT', 'OMS-MT')):
        wstatus_cell_coca_pepsi.at[index,'channel'] = 'Supperettes'
        
    if (row['CITYCODE'] <= 2 and row['m'] in ('OLA-TT', 'OME-TT', 'OMM-TT', 'OMS-TT', 'OSM-TT', 'OWS-TT')):
        wstatus_cell_coca_pepsi.at[index,'channel'] = 'Traditional Food'

In [614]:
###менее гранулярный пгр
wstatus_cell_coca_pepsi.loc[wstatus_cell_coca_pepsi['RU_PGR'] == '8', 'pgr'] = 4
wstatus_cell_coca_pepsi.loc[wstatus_cell_coca_pepsi['RU_PGR'].isin(['4', '5']), 'pgr'] = 2
wstatus_cell_coca_pepsi.loc[wstatus_cell_coca_pepsi['RU_PGR'].isin(['6', '7']), 'pgr'] = 3
wstatus_cell_coca_pepsi.loc[wstatus_cell_coca_pepsi['RU_PGR'].isin(['1', '2', '3']), 'pgr'] = 1

In [615]:
# filling m1
wstatus_cell_coca_pepsi['m1'] = ''

for index,row in wstatus_cell_coca_pepsi.iterrows():
    if (row['CITYCODE'] < 25):
        wstatus_cell_coca_pepsi.at[index,'m1'] = str(row['CITYCODE']) + '-' + row['channel']
    
    if (row['CITYCODE'] > 24):    
        wstatus_cell_coca_pepsi.at[index,'m1'] = row['RU_REGION'] + '-' +  str(row['pgr']).replace('.0', '') + '-' + row['channel']

In [616]:
wstatus_cell_coca_pepsi['Final Cell ID'] = wstatus_cell_coca_pepsi['MAS'].str.split('-', n = 2, expand = True)[0]

In [617]:
#temporary use this file
cells_mbds = pd.read_excel('C:/Users/ScMa9004/Documents/RRES/FMCG_Urban_RES_db_Community_File_cut.xlsx')
pr = pd.read_excel(path + 'Panel Russia_for 313.xlsx', sheet_name = 'Panel', skiprows = 2)
pr_for_AP_cha = pr
pr_map = pr[['MAS', 'CELL_ID']].rename(columns = {'MAS': 'cell'}).drop_duplicates()

In [618]:
pr = pd.pivot_table(pr, index = 'CELL_ID', values = 'act stand', aggfunc = np.sum).reset_index(drop = False)

In [619]:
cells_mbds = pd.pivot_table(cells_mbds, index = 'Final Cell ID', values = 'UNIVERSE', aggfunc = np.sum).reset_index(drop = False)
cells_mbds = cells_mbds.rename(columns = {'Final Cell ID': 'CELL_ID'})

In [620]:
cells_mbds = pd.merge(cells_mbds, pr, how = 'left', on = 'CELL_ID')

In [621]:
cells_mbds['Zf'] = cells_mbds['UNIVERSE'] / cells_mbds['act stand']

In [622]:
cells_mbds = cells_mbds[['CELL_ID', 'Zf']]

In [623]:
wstatus_cell_coca_pepsi = pd.merge(wstatus_cell_coca_pepsi, pr_map, how = 'inner', on = 'cell')

In [624]:
wstatus_cell_coca_pepsi = pd.merge(wstatus_cell_coca_pepsi, cells_mbds, how = 'inner', on = ['CELL_ID'])

In [625]:
# creating pivot table for coca & pepsi check
pivot_c_p = pd.pivot_table(wstatus_cell_coca_pepsi, values='Zf', index='m1', columns='excl_c_p', aggfunc=np.sum)

#turning NaN into 0
pivot_c_p[np.isnan(pivot_c_p)] = 0

# adding total
pivot_c_p['total'] = pivot_c_p['coca'] + pivot_c_p['pepsi'] + pivot_c_p['blank']

#calculating coca_panel and pepsi_panel
pivot_c_p['coca_panel'] = pivot_c_p['coca'] / pivot_c_p['total']
pivot_c_p['pepsi_panel'] = pivot_c_p['pepsi'] / pivot_c_p['total']

In [626]:
# reading universe file
universe_c_p = pd.read_excel('C:/Users/ScMa9004/Scripts/universe_coca_pepsi.xlsx')

In [627]:
# vlookup universe to pivot table
pivot_c_p = pd.merge(pivot_c_p, universe_c_p, how = 'left', on = ['m1'])

In [628]:
# coca check
for index,row in pivot_c_p.iterrows():
    if (row['coca_panel'] < (row['coca_universe'] - 0.05)):
        pivot_c_p.at[index,'coca_check'] = 'too little'
         
    if (row['coca_panel'] >= (row['coca_universe'] - 0.05)) and (row['coca_panel'] <= (row['coca_universe'] + 0.05)):
        pivot_c_p.at[index,'coca_check'] = 'neutral'
         
    if (row['coca_panel'] > (row['coca_universe'] + 0.05)):
        pivot_c_p.at[index,'coca_check'] = 'coca_over'

# pepsi check
for index,row in pivot_c_p.iterrows():
    if (row['pepsi_panel'] < (row['pepsi_universe'] - 0.05)):
        pivot_c_p.at[index,'pepsi_check'] = 'too little'
         
    if (row['pepsi_panel'] >= (row['pepsi_universe'] - 0.05)) and (row['pepsi_panel'] <= (row['pepsi_universe'] + 0.05)):
        pivot_c_p.at[index,'pepsi_check'] = 'neutral'
         
    if (row['pepsi_panel'] > (row['pepsi_universe'] + 0.05)):
        pivot_c_p.at[index,'pepsi_check'] = 'pepsi_over'

# saving pivot table         
writer = pd.ExcelWriter('pivot_c_p.xlsx')
pivot_c_p.to_excel(writer,'Sheet1')
writer.save()

# selecting columns from pivot table with check results
pivot_c_p = pivot_c_p[['m1', 'coca_check', 'pepsi_check']]

#vlookup check results from the pivot table to wstatus_cell_excl_coca_pepsi
wstatus_cell_coca_pepsi = pd.merge(wstatus_cell_coca_pepsi, pivot_c_p, how='left', on=['m1'])

for index,row in wstatus_cell_coca_pepsi.iterrows():
    if ((row['excl_c_p'] == 'blank') or (row['excl_c_p'] == 'pepsi')):
        wstatus_cell_coca_pepsi.at[index,'coca_check'] = ''
    if ((row['excl_c_p'] == 'blank') or (row['excl_c_p'] == 'coca')):
        wstatus_cell_coca_pepsi.at[index,'pepsi_check'] = ''

writer = pd.ExcelWriter('wstatus_cell_coca_pepsi.xlsx')
wstatus_cell_coca_pepsi.to_excel(writer,'Sheet1')
writer.save()

# vlookup coca_check and pepsi_check to tracking system
wstatus_cell_coca_pepsi_result = wstatus_cell_coca_pepsi[['outlet', 'coca_check', 'pepsi_check']]
result = pd.merge(result, wstatus_cell_coca_pepsi_result, how = 'left', on = ['outlet'])

# fill final status for coca & pepsi check
result['status_coca_pepsi'] = ''

for index,row in result.iterrows():
    result.at[index,'status_coca_pepsi'] = 'neutral' 
    if ((row['coca_check'] == 'too little') or (row['pepsi_check'] == 'too little')):
        result.at[index,'status_coca_pepsi'] = 'OK'
    if ((row['coca_check'] == 'coca_over') or (row['pepsi_check'] == 'pepsi_over')):
        result.at[index,'status_coca_pepsi'] = 'No'

# saving tracking system with coca & pepsi check
#writer = pd.ExcelWriter('main.xlsx')
#result.to_excel(writer,'Sheet1')
#writer.save()

In [629]:
result = pd.merge(result, wstatus_cell_coca_pepsi[['outlet', 'm1']], on = 'outlet', how = 'left')

# BABY

In [630]:
# baby checks

#import pandas as pd
#import numpy as np

# import wp1 and st_rum
#st_rum = pd.read_excel('st_rum_296_1.xls',sheetname='st_rum_296_1')
#wp1 = pd.read_excel('wp1.xlsx',sheetname='wp1')

# select bst = 001 and only OBS from st_rum
st_rum_baby = msr.loc[(msr['INDEX_ID'] == 1000296) & msr['STORE_TYPE'].isin(['OBS'])]

# add columns baby_f and baby_d to st_rum from wp1
wp1_baby = wp1[['OCODE','HL141','HL088']]
wp1_baby = wp1_baby.rename(columns={'OCODE': 'outlet', 'HL141': 'baby_f', 'HL088': 'baby_d'})
wstatus_cell_baby = pd.merge(st_rum_baby, wp1_baby, how = 'left', on = ['outlet'])

In [631]:
wstatus_cell_baby['CELL_ID'] = wstatus_cell_baby['MAS'].str.split('-', n = 2, expand = True)[0].astype('int')

In [632]:
wstatus_cell_baby = pd.merge(wstatus_cell_baby, cells_mbds, how = 'left', on = ['CELL_ID'])

In [633]:
for index,row in wstatus_cell_baby.iterrows():
    wstatus_cell_baby.at[index,'z_baby_f'] = row['Zf'] * row['baby_f']
    wstatus_cell_baby.at[index,'z_baby_d'] = row['Zf'] * row['baby_d']   

In [634]:
###менее гранулярный пгр
wstatus_cell_baby.loc[wstatus_cell_baby['RU_PGR'] == '8', 'pgr'] = 4
wstatus_cell_baby.loc[wstatus_cell_baby['RU_PGR'].isin(['4', '5']), 'pgr'] = 2
wstatus_cell_baby.loc[wstatus_cell_baby['RU_PGR'].isin(['6', '7']), 'pgr'] = 3
wstatus_cell_baby.loc[wstatus_cell_baby['RU_PGR'].isin(['1', '2', '3']), 'pgr'] = 1

In [635]:
wstatus_cell_baby['merge'] = ''

for index,row in wstatus_cell_baby.iterrows():
    if (row['CITYCODE'] < 25):
        wstatus_cell_baby.at[index,'merge'] = str(row['CITYCODE'])
             
    if (row['CITYCODE'] > 24):
        wstatus_cell_baby.at[index,'merge'] = row['RU_REGION'] + '-' + str(row['pgr']).replace('.0', '')

In [636]:
# creating pivot table for baby food & baby drug
pivot_baby_f = pd.pivot_table(wstatus_cell_baby, values='z_baby_f', index='merge', aggfunc=np.sum)

pivot_baby_d = pd.pivot_table(wstatus_cell_baby, values='z_baby_d', index='merge', aggfunc=np.sum)

pivot_baby = pd.pivot_table(wstatus_cell_baby, values='Zf', index='merge', aggfunc=np.sum)

#turning NaN into 0
pivot_baby_f[np.isnan(pivot_baby_f)] = 0
pivot_baby_d[np.isnan(pivot_baby_d)] = 0
pivot_baby[np.isnan(pivot_baby)] = 0

#saving the pivot tables in a separate file
writer = pd.ExcelWriter('pivot_baby_f.xlsx')
pivot_baby_f.to_excel(writer,'Sheet1')
writer.save()

writer = pd.ExcelWriter('pivot_baby_d.xlsx')
pivot_baby_d.to_excel(writer,'Sheet1')
writer.save()

writer = pd.ExcelWriter('pivot_baby.xlsx')
pivot_baby.to_excel(writer,'Sheet1')
writer.save()

#reading the pivot tables from the files
pivot_baby_f = pd.read_excel('pivot_baby_f.xlsx',sheet_name='Sheet1')
pivot_baby_d = pd.read_excel('pivot_baby_d.xlsx',sheet_name='Sheet1')
pivot_baby = pd.read_excel('pivot_baby.xlsx',sheet_name='Sheet1')

#merge pivot tables
pivot_baby_all = pd.merge(pivot_baby_f, pivot_baby_d, how = 'left', on = ['merge'])
pivot_baby_all = pd.merge(pivot_baby_all, pivot_baby, how = 'left', on = ['merge'])

#saving the total pivot table in a separate file
writer = pd.ExcelWriter('pivot_baby_all.xlsx')
pivot_baby_all.to_excel(writer,'Sheet1')
writer.save()

#reading the pivot table from the file
pivot_baby_all = pd.read_excel('pivot_baby_all.xlsx',sheet_name='Sheet1')

# calculating baby_f_panel and baby_d_panel
pivot_baby_all['baby_f_panel'] = pivot_baby_all['z_baby_f'] / pivot_baby_all['Zf']
pivot_baby_all['baby_d_panel'] = pivot_baby_all['z_baby_d'] / pivot_baby_all['Zf']

In [637]:
universe_baby = pd.read_excel('C:/Users/ScMa9004/Scripts/universe_baby.xlsx',sheet_name='Sheet1')

pivot_baby_all = pd.merge(pivot_baby_all, universe_baby, how = 'left', on = ['merge'])

In [638]:
# baby_f check
for index,row in pivot_baby_all.iterrows():
    if (row['baby_f_panel'] < (row['universe_baby_f'] - 0.05)):
        pivot_baby_all.at[index,'baby_f_check'] = 'too little'
         
    if (row['baby_f_panel'] >= (row['universe_baby_f'] - 0.05)) and (row['baby_f_panel'] <= (row['universe_baby_f'] + 0.05)):
        pivot_baby_all.at[index,'baby_f_check'] = 'neutral'
         
    if (row['baby_f_panel'] > (row['universe_baby_f'] + 0.05)):
        pivot_baby_all.at[index,'baby_f_check'] = 'baby_f_over'

In [639]:
# baby_d check
for index,row in pivot_baby_all.iterrows():
    if (row['baby_d_panel'] < (row['universe_baby_d'] - 0.05)):
        pivot_baby_all.at[index,'baby_d_check'] = 'too little'
         
    if (row['baby_d_panel'] >= (row['universe_baby_d'] - 0.05)) and (row['baby_d_panel'] <= (row['universe_baby_d'] + 0.05)):
        pivot_baby_all.at[index,'baby_d_check'] = 'neutral'
         
    if (row['baby_d_panel'] > (row['universe_baby_d'] + 0.05)):
        pivot_baby_all.at[index,'baby_d_check'] = 'baby_d_over'
         
#saving the total pivot table in a separate file
writer = pd.ExcelWriter('pivot_baby_all.xlsx')
pivot_baby_all.to_excel(writer,'Sheet1')
writer.save()

In [640]:
# selecting columns from pivot table with check results
pivot_baby_all = pivot_baby_all[['merge', 'baby_f_check', 'baby_d_check']]

In [641]:
#vlookup check results from the pivot table to wstatus_cell_baby
wstatus_cell_baby = pd.merge(wstatus_cell_baby, pivot_baby_all, how='left', on=['merge'])

In [642]:
for index,row in wstatus_cell_baby.iterrows():
    if (row['baby_f'] == 0):
        wstatus_cell_baby.at[index,'baby_f_check'] = 'neutral'
    if (row['baby_d'] == 0):
        wstatus_cell_baby.at[index,'baby_d_check'] = 'neutral'

writer = pd.ExcelWriter('wstatus_cell_baby.xlsx')
wstatus_cell_baby.to_excel(writer,'Sheet1')
writer.save()

# vlookup baby_f_check and baby_d_check to tracking system
wstatus_cell_baby_result = wstatus_cell_baby[['outlet', 'baby_f_check', 'baby_d_check']]
result = pd.merge(result, wstatus_cell_baby_result, how = 'left', on = ['outlet'])

# fill final status for baby_f & baby_d check
result['status_baby'] = ''

for index,row in result.iterrows():
    result.at[index,'status_baby'] = 'neutral' 
    if ((row['baby_f_check'] == 'too little') or (row['baby_d_check'] == 'too little')):
         result.at[index,'status_baby'] = 'OK'
    if ((row['baby_f_check'] == 'baby_f_over') or (row['baby_d_check'] == 'baby_d_over')):
         result.at[index,'status_baby'] = 'No'

# CHA CHECKS

In [643]:
# import universe for cha checks
cha_universe_fmcg = pd.read_excel('C:/Users/ScMa9004/Scripts/cha_check_universe.xlsx',sheet_name='fmcg')   
cha_universe_cig = pd.read_excel('C:/Users/ScMa9004/Scripts/cha_check_universe.xlsx',sheet_name='cig')
cha_universe_beer = pd.read_excel('C:/Users/ScMa9004/Scripts/cha_check_universe.xlsx',sheet_name='beer')

In [644]:
# creating pivot tables for madras cells
pivot_universe_fmcg = pd.pivot_table(cha_universe_fmcg, values='NUHTRP Final', index='Final Cell ID', aggfunc=np.sum)
pivot_universe_cig = pd.pivot_table(cha_universe_cig, values='NUHTRP Final', index='Final Cell ID', aggfunc=np.sum)
pivot_universe_beer = pd.pivot_table(cha_universe_beer, values='NUHTRP Final', index='Final Cell ID', aggfunc=np.sum)
pivot_universe_fmcg = pivot_universe_fmcg.rename(columns = {'NUHTRP Final': 'Sum of universe fmcg'})
pivot_universe_cig = pivot_universe_cig.rename(columns = {'NUHTRP Final': 'Sum of universe cig'})
pivot_universe_beer = pivot_universe_beer.rename(columns = {'NUHTRP Final': 'Sum of universe beer'})

In [645]:
#saving the pivot tables in separate files
writer = pd.ExcelWriter('pivot_universe_fmcg.xlsx')
pivot_universe_fmcg.to_excel(writer,'Sheet1')
writer.save()
writer = pd.ExcelWriter('pivot_universe_cig.xlsx')
pivot_universe_cig.to_excel(writer,'Sheet1')
writer.save()
writer = pd.ExcelWriter('pivot_universe_beer.xlsx')
pivot_universe_beer.to_excel(writer,'Sheet1')
writer.save()

#reading the pivot table from the file
pivot_universe_fmcg = pd.read_excel('pivot_universe_fmcg.xlsx',sheet_name='Sheet1')
pivot_universe_cig = pd.read_excel('pivot_universe_cig.xlsx',sheet_name='Sheet1')
pivot_universe_beer = pd.read_excel('pivot_universe_beer.xlsx',sheet_name='Sheet1')

In [646]:
result.loc[result['cha'].isna(), 'cha'] = 'Standalone'
result.loc[result['INDEX_ID'] == 1000296, 'key_fmcg'] = result['MAS'].str.split('-', n = 2, expand = True)[0].astype('str') + '-' + result['cha'].astype('str')
result.loc[result['INDEX_ID'] == 1000334, 'key_cig'] = result['MAS'].str.split('-', n = 2, expand = True)[0].astype('str') + '-' + result['cha'].astype('str')
result.loc[result['INDEX_ID'] == 1000328, 'key_beer'] = result['MAS'].str.split('-', n = 2, expand = True)[0].astype('str') + '-' + result['cha'].astype('str')

In [647]:
cha_universe_fmcg['key_fmcg'] = cha_universe_fmcg['Final Cell ID'].astype('str') + '-' + cha_universe_fmcg['cha'].astype('str')
cha_universe_cig['key_cig'] = cha_universe_cig['Final Cell ID'].astype('str') + '-' + cha_universe_cig['cha'].astype('str')
cha_universe_beer['key_beer'] = cha_universe_beer['Final Cell ID'].astype('str') + '-' + cha_universe_beer['cha'].astype('str')

In [648]:
#select columns from cha_universe
cha_universe_fmcg = cha_universe_fmcg[['key_fmcg', 'NUHTRP Final']]
cha_universe_cig = cha_universe_cig[['key_cig', 'NUHTRP Final']]
cha_universe_beer = cha_universe_beer[['key_beer', 'NUHTRP Final']]
cha_universe_fmcg = cha_universe_fmcg.rename(columns = {'NUHTRP Final': 'Sum of universe fmcg'})
cha_universe_cig = cha_universe_cig.rename(columns = {'NUHTRP Final': 'Sum of universe cig'})
cha_universe_beer = cha_universe_beer.rename(columns = {'NUHTRP Final': 'Sum of universe beer'})

In [649]:
cha_universe_fmcg = pd.pivot_table(cha_universe_fmcg, index = 'key_fmcg', values = 'Sum of universe fmcg', aggfunc = np.sum).reset_index()
cha_universe_cig = pd.pivot_table(cha_universe_cig, index = 'key_cig', values = 'Sum of universe cig', aggfunc = np.sum).reset_index()
cha_universe_beer = pd.pivot_table(cha_universe_beer, index = 'key_beer', values = 'Sum of universe beer', aggfunc = np.sum).reset_index()

In [650]:
result['Final Cell ID'] = result['MAS'].str.split('-', n = 2, expand = True)[0].replace(np.nan, 0).astype('int')

In [651]:
# vlookup cha & cell universe to final file 
result = pd.merge(result, pivot_universe_fmcg, how = 'left', on = ['Final Cell ID'])
result = pd.merge(result, pivot_universe_cig, how = 'left', on = ['Final Cell ID'])
result = pd.merge(result, pivot_universe_beer, how = 'left', on = ['Final Cell ID'])

In [652]:
result = pd.merge(result, cha_universe_fmcg, how = 'left', on = ['key_fmcg'])
result = pd.merge(result, cha_universe_cig, how = 'left', on = ['key_cig'])
result = pd.merge(result, cha_universe_beer, how = 'left', on = ['key_beer'])

In [653]:
writer = pd.ExcelWriter('main.xlsx')
result.to_excel(writer,'Sheet1')
writer.save()

In [654]:
for index,row in result.iterrows():
    if math.isnan(row['Sum of universe fmcg_x']):
        result.at[index, 'Sum of universe fmcg_x'] = '0'
    if math.isnan(row['Sum of universe cig_x']):
        result.at[index, 'Sum of universe cig_x'] = '0'
    if math.isnan(row['Sum of universe beer_x']):
        result.at[index, 'Sum of universe beer_x'] = '0'
    if math.isnan(row['Sum of universe fmcg_y']):
        result.at[index, 'Sum of universe fmcg_y'] = '0'
    if math.isnan(row['Sum of universe cig_y']):
        result.at[index, 'Sum of universe cig_y'] = '0'
    if math.isnan(row['Sum of universe beer_y']):
        result.at[index, 'Sum of universe beer_y'] = '0'

result = result.rename(columns={'Sum of universe fmcg_x': 'universe fmcg', 'Sum of universe cig_x': 'universe cig', 'Sum of universe beer_x': 'universe beer'})
result = result.rename(columns={'Sum of universe fmcg_y': 'universe fmcg cha', 'Sum of universe cig_y': 'universe cig cha', 'Sum of universe beer_y': 'universe beer cha'})

In [655]:
AP_OP = pd.read_excel('C:/Users/ScMa9004/Scripts/OP.xlsx')   

# AP_cha_micro
pc = pc[pc['ST'] != 'P']

pc.loc[pc['mt'] == False, 'mt'] = 'TT'
pc.loc[pc['mt'] == True, 'mt'] = 'MT'
#pc = pc[pc['chainact'] == 1]

pc['key'] = pc['citycode'].astype('str') + ' ' + pc['type1'] + ' ' + pc['mt'] + ' ' + pc['hip'].astype('str').str.replace('nan', '') + ' '

wrk = pd.merge(pc[['cha', 'key', 'chainact']], pr_for_AP_cha[['key', 'MAS']], how = 'inner')
wrk['cha'] = wrk['cha'].fillna('nan')
AP_cha_fmcg = pd.pivot_table(wrk, index = ['MAS', 'key', 'cha'],
                             values = 'chainact', 
                             aggfunc = 'count').reset_index(drop = False)
AP_cha_fmcg = AP_cha_fmcg[AP_cha_fmcg['MAS'] != 0]
AP_cha_fmcg['key_cha'] = AP_cha_fmcg['MAS'].astype('int').astype('str') + '-' + AP_cha_fmcg['cha']
AP_cha_fmcg = AP_cha_fmcg.rename(columns = {'MAS': 'cell', 'chainact': 'AP_cha_fmcg'})
ap_cha_fmcg = AP_cha_fmcg[['key_cha', 'AP_cha_fmcg']]
ap_cha_fmcg = pd.pivot_table(ap_cha_fmcg, index = 'key_cha', values = 'AP_cha_fmcg', aggfunc = np.sum).reset_index()

wrk = pd.merge(pc[['cha', 'key', 'chainact']], pr_for_AP_cha[['key', 'MAS.1']], how = 'inner')
wrk['cha'] = wrk['cha'].fillna('nan')
AP_cha_cig = pd.pivot_table(wrk, index = ['MAS.1', 'key', 'cha'],
                            values = 'chainact',
                            aggfunc = 'count').reset_index(drop = False)
AP_cha_cig = AP_cha_cig[AP_cha_cig['MAS.1'] != 0]
AP_cha_cig['key_cha'] = AP_cha_cig['MAS.1'].astype('int').astype('str') + '-' + AP_cha_cig['cha']
AP_cha_cig = AP_cha_cig.rename(columns = {'MAS.1': 'cell', 'chainact': 'AP_cha_cig'})
ap_cha_cig = AP_cha_cig[['key_cha', 'AP_cha_cig']]
ap_cha_cig = pd.pivot_table(ap_cha_cig, index = 'key_cha', values = 'AP_cha_cig', aggfunc = np.sum).reset_index()

wrk = pd.merge(pc[['cha', 'key', 'chainact']], pr_for_AP_cha[['key', 'MAS.2']], how = 'inner')
wrk['cha'] = wrk['cha'].fillna('nan')
AP_cha_beer = pd.pivot_table(wrk, index = ['MAS.2', 'key', 'cha'],
                             values = 'chainact', 
                             aggfunc = np.sum).reset_index(drop = False)
AP_cha_beer = AP_cha_beer[AP_cha_beer['MAS.2'] != 0]
AP_cha_beer['key_cha'] = AP_cha_beer['MAS.2'].astype('int').astype('str') + '-' + AP_cha_beer['cha']
AP_cha_beer = AP_cha_beer.rename(columns = {'MAS.2': 'cell', 'chainact': 'AP_cha_beer'})
ap_cha_beer = AP_cha_beer[['key_cha', 'AP_cha_beer']]
ap_cha_beer = pd.pivot_table(ap_cha_beer, index = 'key_cha', values = 'AP_cha_beer', aggfunc = np.sum).reset_index()

In [657]:
# AP_cha_madras

#Колонка должна называться ST, но иногда она без названия
pc = pc[pc['ST'] != 'P']
#pc = pc[pc['Unnamed: 183'] != 'P']


pc.loc[pc['mt'] == False, 'mt'] = 'TT'
pc.loc[pc['mt'] == True, 'mt'] = 'MT'
#pc = pc[pc['chainact'] == 1]

pc['key'] = pc['citycode'].astype('str') + ' ' + pc['type1'] + ' ' + pc['mt'] + ' ' + pc['hip'].astype('str').str.replace('nan', '') + ' '

wrk = pd.merge(pc[['cha', 'key', 'chainact']], pr_for_AP_cha[['key', 'CELL_ID']], how = 'inner')
wrk['cha'] = wrk['cha'].fillna('nan')
AP_cha_fmcg = pd.pivot_table(wrk, index = ['CELL_ID', 'key', 'cha'],
                             values = 'chainact', 
                             aggfunc = 'count').reset_index(drop = False)
AP_cha_fmcg = AP_cha_fmcg[AP_cha_fmcg['CELL_ID'] != 0]
AP_cha_fmcg['key_cha'] = AP_cha_fmcg['CELL_ID'].astype('int').astype('str') + '-' + AP_cha_fmcg['cha']
AP_cha_fmcg = AP_cha_fmcg.rename(columns = {'CELL_ID': 'cell', 'chainact': 'AP_cha_fmcg'})
ap_cha_fmcg = AP_cha_fmcg[['key_cha', 'AP_cha_fmcg']]
ap_cha_fmcg = pd.pivot_table(ap_cha_fmcg, index = 'key_cha', values = 'AP_cha_fmcg', aggfunc = np.sum).reset_index()

wrk = pd.merge(pc[['cha', 'key', 'chainact']], pr_for_AP_cha[['key', 'CELL_ID.1']], how = 'inner')
wrk['cha'] = wrk['cha'].fillna('nan')
AP_cha_cig = pd.pivot_table(wrk, index = ['CELL_ID.1', 'key', 'cha'],
                            values = 'chainact',
                            aggfunc = 'count').reset_index(drop = False)
AP_cha_cig = AP_cha_cig[AP_cha_cig['CELL_ID.1'] != 0]
AP_cha_cig['key_cha'] = AP_cha_cig['CELL_ID.1'].astype('int').astype('str') + '-' + AP_cha_cig['cha']
AP_cha_cig = AP_cha_cig.rename(columns = {'CELL_ID.1': 'cell', 'chainact': 'AP_cha_cig'})
ap_cha_cig = AP_cha_cig[['key_cha', 'AP_cha_cig']]
ap_cha_cig = pd.pivot_table(ap_cha_cig, index = 'key_cha', values = 'AP_cha_cig', aggfunc = np.sum).reset_index()

wrk = pd.merge(pc[['cha', 'key', 'chainact']], pr_for_AP_cha[['key', 'CELL_ID.2']], how = 'inner')
wrk['cha'] = wrk['cha'].fillna('nan')
AP_cha_beer = pd.pivot_table(wrk, index = ['CELL_ID.2', 'key', 'cha'],
                             values = 'chainact', 
                             aggfunc = np.sum).reset_index(drop = False)
AP_cha_beer = AP_cha_beer[AP_cha_beer['CELL_ID.2'] != 0]
AP_cha_beer['key_cha'] = AP_cha_beer['CELL_ID.2'].astype('int').astype('str') + '-' + AP_cha_beer['cha']
AP_cha_beer = AP_cha_beer.rename(columns = {'CELL_ID.2': 'cell', 'chainact': 'AP_cha_beer'})
ap_cha_beer = AP_cha_beer[['key_cha', 'AP_cha_beer']]
ap_cha_beer = pd.pivot_table(ap_cha_beer, index = 'key_cha', values = 'AP_cha_beer', aggfunc = np.sum).reset_index()

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [658]:
result['cha'] = result['cha'].str.replace('Standalone', 'nan')
#result['key_cha'] = result['cell'].astype('str') + '-' + result['cha']
result['key_cha'] = result['Final Cell ID'].astype('str') + '-' + result['cha']

In [659]:
# vlookup OP and AP_cha    
result = pd.merge(result, AP_OP, how = 'left', on = ['cell'])
result = pd.merge(result, ap_cha_fmcg, how = 'left', on = ['key_cha'])
result = pd.merge(result, ap_cha_cig, how = 'left', on = ['key_cha'])
result = pd.merge(result, ap_cha_beer, how = 'left', on = ['key_cha'])

In [660]:
writer = pd.ExcelWriter('main.xlsx')
result.to_excel(writer,'Sheet1')
writer.save()

for index,row in result.iterrows():
    if math.isnan(row['AP_fmcg']):
        result.loc[index, 'AP_fmcg'] = 0
    if math.isnan(row['AP_cig']):
        result.loc[index, 'AP_cig'] = 0
    if math.isnan(row['AP_beer']):
        result.loc[index, 'AP_beer'] = 0
    if math.isnan(row['AP_fmcg']):
        result.loc[index, 'AP_cha_fmcg'] = 0
    if math.isnan(row['AP_cha_cig']):
        result.loc[index, 'AP_cha_cig'] = 0
    if math.isnan(row['AP_cha_beer']):
        result.loc[index, 'AP_cha_beer'] = 0

In [661]:
result.loc[result['universe cig'] == 0, 'universe cig'] = np.nan
result.loc[result['universe beer'] == 0, 'universe beer'] = np.nan
result.loc[result['universe fmcg'] == 0, 'universe fmcg'] = np.nan

In [662]:
# calculate possible_no_cha
#for index,row in result.iterrows():
#    if (row['bst'] == 1 and isinstance(row['cha'], str)):
#        if (row['universe fmcg cha']/row['universe fmcg']*row['OP_fmcg'] < 1):
#            result.at[index, 'possible_fmcg'] = 1
#        else:
#            result.at[index, 'possible_fmcg'] = row['universe fmcg cha']/row['universe fmcg']*row['OP_fmcg']
#    writer = pd.ExcelWriter('main.xlsx')
#    result.to_excel(writer,'Sheet1')
#    writer.save()
#
#    if ((row['bst'] == 3 or row['bst'] == 501 or row['bst'] == 502) and isinstance(row['cha'], str)):
#        if (row['universe cig cha']/row['universe cig']*row['OP_cig'] < 1):
#            result.at[index, 'possible_cig'] = 1
#        else:
#            result.at[index, 'possible_cig'] = row['universe cig cha']/row['universe cig']*row['OP_cig']
#    writer = pd.ExcelWriter('main.xlsx')
#    result.to_excel(writer,'Sheet1')
#    writer.save()
#    
#    if ((row['bst'] == 4 or row['bst'] == 501) and isinstance(row['cha'], str)):
#        if (row['universe beer cha']/row['universe beer']*row['OP_beer'] < 1):
#            result.at[index, 'possible_beer'] = 1
#        else:
#            result.at[index, 'possible_beer'] = row['universe beer cha']/row['universe beer']*row['OP_beer']
#    writer = pd.ExcelWriter('main.xlsx')
#    result.to_excel(writer,'Sheet1')
#    writer.save()   
#

In [663]:
# calculate possible_no_cha
for index,row in result.iterrows():
    if (row['INDEX_ID'] == 1000296 and isinstance(row['cha'], str)):
        if (row['universe fmcg cha']/row['universe fmcg']*row['AP_fmcg'] < 1):
            result.at[index, 'possible_fmcg'] = 1
        else:
            result.at[index, 'possible_fmcg'] = row['universe fmcg cha']/row['universe fmcg']*row['AP_fmcg']
#    writer = pd.ExcelWriter('main.xlsx')
#    result.to_excel(writer,'Sheet1')
#    writer.save()

    if (row['INDEX_ID'] == 1000334 and isinstance(row['cha'], str)):
        if (row['universe cig cha']/row['universe cig']*row['AP_cig'] < 1):
            result.at[index, 'possible_cig'] = 1
        else:
            result.at[index, 'possible_cig'] = row['universe cig cha']/row['universe cig']*row['AP_cig']
#    writer = pd.ExcelWriter('main.xlsx')
#    result.to_excel(writer,'Sheet1')
#    writer.save()
    
    if (row['INDEX_ID'] == 1000328 and isinstance(row['cha'], str)):
        if (row['universe beer cha']/row['universe beer']*row['AP_beer'] < 1):
            result.at[index, 'possible_beer'] = 1
        else:
            result.at[index, 'possible_beer'] = row['universe beer cha']/row['universe beer']*row['AP_beer']
#    writer = pd.ExcelWriter('main.xlsx')
#    result.to_excel(writer,'Sheet1')
#    writer.save()   


In [664]:
for index,row in result.iterrows():
    if math.isnan(row['possible_fmcg']):
        result.at[index, 'possible_fmcg'] = 0
    if math.isnan(row['possible_cig']):
        result.at[index, 'possible_cig'] = 0
    if math.isnan(row['possible_beer']):
        result.at[index, 'possible_beer'] = 0 
        
for index,row in result.iterrows():
    result.at[index,'possible_fmcg'] = round(row['possible_fmcg'],0)
    result.at[index,'possible_cig'] = round(row['possible_cig'],0)
    result.at[index,'possible_beer'] = round(row['possible_beer'],0)

In [665]:
result['cha'] = result['cha'].replace('nan', np.nan)

In [666]:
# ДЕЛЕНИЕ НА НОЛЬ
result.loc[(result['AP_fmcg'] == 0) & (result['AP_cha_fmcg'] == 0), 'AP_fmcg'] = 9999
result.loc[(result['AP_cig'] == 0) & (result['AP_cha_cig'] == 0), 'AP_cig'] = 9999
result.loc[(result['AP_beer'] == 0) & (result['AP_cha_beer'] == 0), 'AP_beer'] = 9999

for index,row in result.iterrows():
    if ((row['INDEX_ID'] == 1000296) and (row['bst'] == 1) and isinstance(row['cha'], str)):    
        if ((row['AP_cha_fmcg']/row['AP_fmcg'] <= 0.5) or (row['AP_cha_fmcg'] == 1) or (row['AP_cha_fmcg'] <= row['possible_fmcg'])) :
            result.at[index, 'status cha'] = 'OK'
        else:
            result.at[index, 'status cha'] = 'no'
    
    if ((row['INDEX_ID'] == 1000334) and ((row['bst'] == 3) or (row['bst'] == 501) or (row['bst'] == 502)) and isinstance(row['cha'], str)):    
        if ((row['AP_cha_cig']/row['AP_cig'] <= 0.5) or (row['AP_cha_cig'] == 1) or (row['AP_cha_cig'] <= row['possible_cig'])):
            result.at[index, 'status cha'] = 'OK'
        else:
            result.at[index, 'status cha'] = 'no'
    
    if ((row['INDEX_ID'] == 1000328) and (row['bst'] == 4) and isinstance(row['cha'], str)):    
        if ((row['AP_cha_beer']/row['AP_beer'] <= 0.5) or (row['AP_cha_beer'] == 1) or (row['AP_cha_beer'] <= row['possible_beer'])):
            result.at[index, 'status cha'] = 'OK'
        else:
            result.at[index, 'status cha'] = 'no'
            
    if ((row['INDEX_ID'] == 1000328) and (row['bst'] == 501) and isinstance(row['cha'], str)):
        result.at[index, 'status cha'] = result['status cha'][(result['outlet'] == row['outlet']) & (result['INDEX_ID'] == 1000334)].iloc[0]
        
    if (((row['INDEX_ID'] == 1000328) | (row['INDEX_ID'] == 1000334)) and (row['bst'] == 1) and isinstance(row['cha'], str)):
        result.at[index, 'status cha'] = result['status cha'][(result['outlet'] == row['outlet']) & (result['INDEX_ID'] == 1000296)].iloc[0]

#for index,row in result.iterrows():
#    if (row['bst'] == 1 and isinstance(row['cha'], str) and (row['cell'] > 20000 and row['cell'] < 50000)): 
#        result.at[index, 'status cha'] = 'OK'
#    if ((row['bst'] == 3 or row['bst'] == 501 or row['bst'] == 502) and isinstance(row['cha'], str) and (row['cell_c'] > 20000 and row['cell_c'] < 50000)): 
#        result.at[index, 'status cha'] = 'OK'
#    if ((row['bst'] == 4 or row['bst'] == 501) and isinstance(row['cha'], str) and (row['cell_b'] > 20000 and row['cell_b'] < 50000)): 
#        result.at[index, 'status cha'] = 'OK'
        
writer = pd.ExcelWriter('main.xlsx')
result.to_excel(writer,'Sheet1')
writer.save()

In [667]:
result.loc[(result['AP_fmcg'] == 9999), 'AP_fmcg'] = 0
result.loc[(result['AP_cig'] == 9999), 'AP_cig'] = 0
result.loc[(result['AP_beer'] == 9999), 'AP_beer'] = 0

In [668]:
########вот здесь фулл документед инфо и другое смотреть проставляется не тот статус
# fill column c_p
for index,row in result.iterrows():
    result.at[index,'c_p'] = 0 
    if (row['status_coca_pepsi'] == 'OK'):
         result.at[index,'c_p'] = 1
    if (row['status_coca_pepsi'] == 'No'):
         result.at[index,'c_p'] = -1
         
# fill column baby
for index,row in result.iterrows():
    result.at[index,'baby'] = 0 
    if (row['status_baby'] == 'OK'):
         result.at[index,'baby'] = 1
    if (row['status_baby'] == 'No'):
         result.at[index,'baby'] = -1
         
# fill column total
for index,row in result.iterrows():
    result.at[index,'total'] = 0 
    if (row['c_p'] + row['baby'] > 0):
         result.at[index,'total'] = 1
    if (row['c_p'] + row['baby'] < 0):
         result.at[index,'total'] = -1        
         
# fill column final_status_microrep
for index,row in result.iterrows():
    result.at[index,'final_status_microrep'] = 'neutral' 
    if (row['total'] > 0):
         result.at[index,'final_status_microrep'] = 'OK'
    if (row['total'] < 0):
         result.at[index,'final_status_microrep'] = 'not'            
         
# fill column merge_data_quality         
for index,row in result.iterrows():
    result.at[index,'merge_data_quality'] = str(row['Interval for info']) + '-' + str(row['Source of info']) + '-' + str(row['Type of info']) 
             
# fill column merge_panel_quality         
for index,row in result.iterrows():
    result.at[index,'merge_panel_quality'] = row['final_status_microrep'] 

# fill column data_quality
for index,row in result.iterrows():
    result.loc[index,'data_quality'] = 'wait' 
    if ((row['merge_data_quality'] == 'Full-documented-sales') or (row['merge_data_quality'] == 'Full-documented-purchase')):
         result.loc[index,'data_quality'] = 'OK'

# fill column panel_quality
for index,row in result.iterrows():
    result.loc[index,'panel_quality'] = 'OK' 
    if (row['final_status_microrep'] == 'not'):
         result.loc[index,'panel_quality'] = 'No'

In [669]:
# fill final status
result['final_status'] = result['status cha'] + '-' + result['data_quality'] + '-' + result['panel_quality']

for index,row in result.iterrows():
    result.at[index,'final_status_2'] = 'no' 
    if (row['final_status'] == 'OK-OK-OK'):
         result.at[index,'final_status_2'] = 'OK'    
    if (row['final_status'] == 'OK-wait-OK'):
         result.at[index,'final_status_2'] = 'wait'       

In [670]:
# до этого шага один индекс = одна строка, после - одна тт = одна строка.
final = pd.DataFrame(columns = ['outlet', 'pno', 'm1',
                                #'region DA', 
                                'type', 'Type of info', 'Source of info',
       'Interval for info', 'Comments for DA', 'bst', 'Outlet',
       #'SMS_ID', 'STORE_TYPE', 
                                'TRADE_TYPE', 'PROVINCE', 'SALES_AREA',
       'HL156', 'HL013', 'AUTORIZED', 'cha', 
                                #'SMS_SERVICE',
                                'CITYCODE',
       #'RU_REGION', 'RU_RGN', 'RU_CCN', 'NOT_NEED_CHA', 'NEED_CCODE', 
                                'RU_PGR',
       #'RU_SUBJECT', 'RGN_BEER', 'CCN_BEER', 'REGION_CIG', 'RGN_CIG',
       #'CCN_CIG', 'DIST_STRATA', 'RU_CHA', 
                                'cell_fmcg', 'cell_cig', 'cell_beer', 'status cha',
       #'Chainact', 
                                'om', 'coca_check', 'pepsi_check', 'status_coca_pepsi',
       'baby_f_check', 'baby_d_check', 'status_baby', 
        #                        'key_fmcg', 'key_cig', 'key_beer', 
        'Final Cell ID FMCG', 'Final Cell ID CIG', 'Final Cell ID BEER',
        'universe fmcg', 'universe cig',
       'universe beer', 'universe fmcg cha', 'universe cig cha',
       'universe beer cha',
        #                        'key_cha',
        'AP_fmcg', 'AP_cig', 'AP_beer',
                                
       'AP_cha_fmcg', 'AP_cha_cig', 'AP_cha_beer',
       'possible_fmcg', 'possible_cig', 'possible_beer', 'c_p', 'baby',
       'total', 'final_status_microrep', 'merge_data_quality',
       'merge_panel_quality', 'data_quality', 'panel_quality', 'final_status',
       'final_status_2'])

columns = ['outlet', 'pno', 'm1',
           'type', 'Type of info', 'Source of info',
       'Interval for info', 'Comments for DA', 'bst', 'Outlet',
        'TRADE_TYPE', 'PROVINCE', 'SALES_AREA',
       'HL156', 'HL013', 'AUTORIZED', 'cha', 'CITYCODE',
       'RU_PGR',
       #### 'cell_fmcg', 'cell_cig', 'cell_beer',
       'status cha', 'om', 'coca_check', 'pepsi_check', 'status_coca_pepsi',
       'baby_f_check', 'baby_d_check', 'status_baby', 
       #'key_cha',
       #### 'key_fmcg', 'key_cig',       'key_beer', 
       #### 'Final Cell ID FMCG', 'Final Cell ID CIG', 'Final Cell ID BEER',
       #'universe fmcg', 'universe cig',
       #'universe beer', 'universe fmcg cha', 'universe cig cha',
       #'universe beer cha', 'AP_fmcg', 'AP_cig', 'AP_beer', 'AP_cha_fmcg', 'AP_cha_cig', 'AP_cha_beer',
       #'possible_fmcg', 'possible_cig', 'possible_beer',
       'c_p', 'baby',
       'total', 'final_status_microrep', 'merge_data_quality',
       'merge_panel_quality', 'data_quality', 'panel_quality', 'final_status',
       'final_status_2']

# здесть те колонки, что в предыдущем списке стоят после решетки (суммируем значения в колнке для одной и той же тт)
shmolumns = ['universe fmcg', 'universe cig', 'universe beer', 'universe fmcg cha', 'universe cig cha',
             'universe beer cha', 'AP_fmcg', 'AP_cig', 'AP_beer',
             'AP_cha_fmcg', 'AP_cha_cig', 'AP_cha_beer',
             'possible_fmcg', 'possible_cig', 'possible_beer']

l = -1
for k in result['outlet'].unique():
    l += 1
    for m in columns:
        try:
            final.loc[l, m] = result[result['outlet'] == k][m].unique().item()
        except:
            print(m, k, 'для одной и той же тт в разных индексах стоят разные значения параметра, которые должны быть одинаковыми')
    for n in shmolumns:
        final.loc[l, n] = result[result['outlet'] == k][n].sum()
    # здесь обрабатываются колонки, помеченные четырьмя решетками
    for i, j in result[result['outlet'] == k].iterrows():
        if j['INDEX_ID'] == 1000296:
            final.loc[l, 'cell_fmcg'] = j['cell']
            #final.loc[l, 'key_fmcg'] = j['key_fmcg']
            final.loc[l, 'Final Cell ID FMCG'] = j['Final Cell ID']
        if j['INDEX_ID'] == 1000334:
            final.loc[l, 'cell_cig'] = j['cell']
            #final.loc[l, 'key_cig'] = j['key_cig']
            final.loc[l, 'Final Cell ID CIG'] = j['Final Cell ID']
        if j['INDEX_ID'] == 1000328:
            final.loc[l, 'cell_beer'] = j['cell']
            #final.loc[l, 'key_beer'] = j['key_beer']
            final.loc[l, 'Final Cell ID BEER'] = j['Final Cell ID']

In [671]:
# saving tracking system with all checks (except for cha)
writer = pd.ExcelWriter('main.xlsx')
final.to_excel(writer, 'Sheet1', index = False)
writer.save()